In [119]:
##IMPORT DEPENDENCIES##
#gym stuff#
import gym
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete

#helpers#
import numpy as np
import random
import os

#stablebaselines#
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3. common.evaluation import evaluate_policy

#data plitting#
from sklearn.model_selection import train_test_split

In [120]:
# Define a mapping of body parts to column ranges
body_parts_mapping = {
    'Head': (0, 1),
    'SpineShoulders': (3, 4),
    'SpineMid': (6, 7),
    'LeftShoulder': (9, 10),
    'RightShoulder': (12, 13),
    'LeftElbow': (15, 16),
    'RightElbow': (18, 19),
    'LeftWrist': (21, 22),
    'RightWrist': (24, 25),
    'SpineBase': (27, 28),
    'LeftHip': (30, 31),
    'RightHip': (33, 34),
    'LeftKnee': (36, 37),
    'RightKnee': (39, 40),
    'LeftAnkle': (42, 43),
    'RightAnkle': (45, 46),
}

In [121]:
data_folder = os.path.join('data_copy')

In [122]:
data_folder

'data_copy'

In [123]:
# Step 1: Preprocess Data
# Updated Step 1: Preprocess Data
def preprocess_data(data_folder):
    joint_positions = []
    actions = []

    exercise_folders = os.listdir(data_folder)
    exercise_folders.remove('guide')
    exercise_folders.remove('plot') 

    # Define the output folder path
    output_folder = os.path.join("States and Actions")
    os.makedirs(output_folder, exist_ok=True)

    for exercise_folder in exercise_folders:

        exercise_path = os.path.join(data_folder, exercise_folder)
        exercise_path_list = os.listdir(exercise_path)
        
        for typedata_folder in exercise_path_list:
        
            typedata_path = os.path.join(exercise_path, typedata_folder)
            typedata_path_list = os.listdir(typedata_path)

            for typepatient_folder in typedata_path_list:

                typepatient_path = os.path.join(typedata_path, typepatient_folder)
                typepatient_path_list = os.listdir(typepatient_path)

                for patient_folder in typepatient_path_list:

                    patient_path = os.path.join(typepatient_path, patient_folder)
                    patient_path_list = os.listdir(patient_path)

                    for trial_file in patient_path_list:
                        if not trial_file.startswith('.'):  # Skip files/directories starting with a period
                            file_path = os.path.join(patient_path, trial_file)
                            joints_data = read_txt_file(file_path)
        
                            # Extract relevant columns based on body parts mapping
                            extracted_data = extract_body_parts(joints_data)
                            
                            # Print metadata
                            print('-----------------------------------------------')
                            print('Exercise -', exercise_folder)
                            print('Captured / Measured -', typedata_folder)
                            print('Affected / Unaffected -', typepatient_folder)
                            print('Patient -', patient_folder)
                            print('Trial -', trial_file)
                            
                            # Process each instance
                            #instance_idx = 1
                            #for instance_data in extracted_data:
                                #print(f'------Instance {instance_idx}------')
                                #for body_part, body_part_data in instance_data.items():
                                    #print('Joint -', body_part)
                                    #print('Joint Coords -', body_part_data)
                                #instance_idx += 1

                            # Process each instance and generate actions
                            instance_actions = generate_actions(extracted_data)
                            actions.extend(instance_actions)

                            # Save the joint positions
                            joint_positions_output_folder = os.path.join(output_folder, exercise_folder, typedata_folder, typepatient_folder, patient_folder)
                            os.makedirs(joint_positions_output_folder, exist_ok=True)
                            joint_positions_file = os.path.join(joint_positions_output_folder, f"{trial_file}_joint_positions.txt")
                            with open(joint_positions_file, 'w') as file:
                                for instance in extracted_data:
                                    file.write(f"Instance:\n")
                                    for joint, coords in instance.items():
                                        file.write(f"{joint}: {coords}\n")
                                    file.write("\n")
                            
                            # Save the actions as text file
                            actions_output_folder = os.path.join(output_folder, exercise_folder, typedata_folder, typepatient_folder, patient_folder)
                            os.makedirs(actions_output_folder, exist_ok=True)
                            actions_file = os.path.join(actions_output_folder, f"{trial_file}_actions.txt")
                            with open(actions_file, 'w') as file:
                                for action in instance_actions:
                                    file.write(f"{action[0]} {action[1]}\n")

                            joint_positions.append(extracted_data)
    
     # Split data into training and testing sets
    train_data, test_data = train_test_split(joint_positions, test_size=0.2, random_state=42)
    train_actions, test_actions = train_test_split( actions, test_size=0.2, random_state=42)

    print('The End')
    
    return train_data, test_data, train_actions, test_actions

# Function to generate grid for joint coordinates
def generate_grid(joint_coords):
    # Define grid size (adjust as needed)
    grid_size = (10, 10)
    # Determine grid position of the joint coordinates
    grid_x = int(joint_coords[0] * grid_size[0])
    grid_y = int(joint_coords[1] * grid_size[1])
    return grid_x, grid_y

# Function to compare wrist coordinates and generate action
def generate_actions(extracted_data):
    actions = []
    instance_idx = 1  # Track instance number
    for i in range(0, len(extracted_data), 5):
        if i + 5 < len(extracted_data):
            # Extract wrist coordinates for current and next instances
            current_left_wrist = extracted_data[i]['LeftWrist']
            next_left_wrist = extracted_data[i + 5]['LeftWrist']
            current_right_wrist = extracted_data[i]['RightWrist']
            next_right_wrist = extracted_data[i + 5]['RightWrist']
            
            # Generate grid positions for wrist coordinates
            current_left_grid = generate_grid(current_left_wrist)
            next_left_grid = generate_grid(next_left_wrist)
            current_right_grid = generate_grid(current_right_wrist)
            next_right_grid = generate_grid(next_right_wrist)
            
            # Compare grid positions to determine movement direction for left wrist
            left_movement = compare_positions(current_left_grid, next_left_grid)
            # Compare grid positions to determine movement direction for right wrist
            right_movement = compare_positions(current_right_grid, next_right_grid)
            
            # Map grid position comparison to action
            left_action = f"{left_movement}" if left_movement != "Hold" else "Hold"
            right_action = f"{right_movement}" if right_movement != "Hold" else "Hold"
            
            # Print the action for the current instance range
            print(f"Action from instance {instance_idx} to {instance_idx+4}: Left-  {left_action}, Right-  {right_action}")
            
            actions.append((left_action, right_action))
            instance_idx += 5  # Move to the next instance
    return actions

# Function to compare grid positions and determine movement direction
def compare_positions(current_grid, next_grid):
    # Compare grid positions between current and next coordinates
    # Return movement direction based on the comparison result
    # Example implementation (adjust as needed)
    if current_grid[0] == next_grid[0] and current_grid[1] == next_grid[1]:
        return "Hold"
    elif current_grid[0] < next_grid[0]:
        if current_grid[1] < next_grid[1]:
            return "DownRight"
        elif current_grid[1] > next_grid[1]:
            return "UpRight"
        else:
            return "Right"
    elif current_grid[0] > next_grid[0]:
        if current_grid[1] < next_grid[1]:
            return "DownLeft"
        elif current_grid[1] > next_grid[1]:
            return "UpLeft"
        else:
            return "Left"
    else:
        if current_grid[1] < next_grid[1]:
            return "Down"
        elif current_grid[1] > next_grid[1]:
            return "Up"
    return "no movement detected"  # Default action if no movement detected

# Function to map left and right movement directions to action
def map_to_action(left_movement, right_movement):
    # Map left and right movement directions to action
    # Return the corresponding action
    return f"{left_movement}_{right_movement}"


def read_txt_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        joints_data = [list(map(float, line.split(';'))) for line in lines]
    return joints_data

def extract_body_parts(joints_data):
    num_instances = len(joints_data)
    extracted_data = []

    # Iterate over instances
    for instance_idx in range(num_instances):
        instance_data = {}
        # Iterate over body parts
        for body_part, (start_col, end_col) in body_parts_mapping.items():
            body_part_data = joints_data[instance_idx][start_col:end_col + 1]
            instance_data[body_part] = body_part_data
        extracted_data.append(instance_data)

    return extracted_data


In [124]:
train_data, test_data, train_actions, test_actions = preprocess_data(data_folder)

-----------------------------------------------
Exercise - E1
Captured / Measured - capture
Affected / Unaffected - affected
Patient - P01
Trial - E1_filt_captured_P01_affected_1.txt
Action from instance 1 to 5: Left-  Down, Right-  Hold
Action from instance 6 to 10: Left-  Right, Right-  Hold
Action from instance 11 to 15: Left-  UpLeft, Right-  Hold
Action from instance 16 to 20: Left-  Up, Right-  Hold
Action from instance 21 to 25: Left-  Up, Right-  Hold
-----------------------------------------------
Exercise - E1
Captured / Measured - capture
Affected / Unaffected - affected
Patient - P01
Trial - E1_filt_captured_P01_affected_10.txt
Action from instance 1 to 5: Left-  Down, Right-  Hold
Action from instance 6 to 10: Left-  Hold, Right-  Hold
Action from instance 11 to 15: Left-  Up, Right-  Hold
Action from instance 16 to 20: Left-  Hold, Right-  Hold
Action from instance 21 to 25: Left-  Up, Right-  Hold
-----------------------------------------------
Exercise - E1
Captured / M

In [113]:
class PatientHealthEnv(Env):
    def __init__(self, train_data, train_actions, test_data, test_actions):
        self.action_space = Discrete(len(train_actions))  # Action space size is the number of available actions
        self.observation_space = Box(low=0, high=100, shape=(1, len(train_data[0])))  # State space size is the number of joints
        self.train_data = train_data  # Training data containing joint positions
        self.train_actions = train_actions  # Training actions
        self.test_data = test_data  # Testing data containing joint positions
        self.test_actions = test_actions  # Testing actions
        self.num_files = len(train_data)  # Number of files in the dataset

    def step(self, action):
        # Unpack the action tuple for left and right wrists
        left_action, right_action = action
        
        # Calculate reward based on the similarity of actions
        action_scores = []
        for i in range(len(self.train_actions)):
            score = 0
            # Compare actions for the left wrist
            if left_action == self.train_actions[i][0]:
                score -= 1  # Negative reward for each equal pair of actions with affected patients
            if left_action == self.test_actions[i][0]:
                score += 1  # Positive reward for each equal pair of actions with unaffected patients
            # Compare actions for the right wrist
            if right_action == self.train_actions[i][1]:
                score -= 1  # Negative reward for each equal pair of actions with affected patients
            if right_action == self.test_actions[i][1]:
                score += 1  # Positive reward for each equal pair of actions with unaffected patients
            action_scores.append(score)
        reward = sum(action_scores)  # Sum of rewards

        # If the sum of rewards is positive, repeat the process for unaffected patients in the training data
        if reward > 0:
            # Separate the training data and actions into two groups: 'P' (Patient) and 'S' (Subject)
            train_data_P = [data for data in self.train_data if data['Patient'].startswith('P')]
            train_data_S = [data for data in self.train_data if data['Patient'].startswith('S')]
            train_actions_P = [action for idx, action in enumerate(self.train_actions) if self.train_data[idx]['Patient'].startswith('P')]
            train_actions_S = [action for idx, action in enumerate(self.train_actions) if self.train_data[idx]['Patient'].startswith('S')]

            # Repeat the comparison process for unaffected patients starting with 'P'
            if train_data_P:
                for i in range(min(len(train_actions_P), len(self.test_actions))):
                    score = 0
                    # Compare actions for the left wrist
                    if left_action == train_actions_P[i][0]:
                        score -= 1  # Negative reward for each equal pair of actions with affected patients
                    if left_action == self.test_actions[i][0]:
                        score -= 1  # Additional negative reward for each equal pair of actions with unaffected patients
                    # Compare actions for the right wrist
                    if right_action == train_actions_P[i][1]:
                        score -= 1  # Negative reward for each equal pair of actions with affected patients
                    if right_action == self.test_actions[i][1]:
                        score -= 1  # Additional negative reward for each equal pair of actions with unaffected patients
                    action_scores.append(score)
            
            # Repeat the comparison process for unaffected patients starting with 'S'
            if train_data_S:
                for i in range(min(len(train_actions_S), len(self.test_actions))):
                    score = 0
                    # Compare actions for the left wrist
                    if left_action == train_actions_S[i][0]:
                        score += 1  # Positive reward for each equal pair of actions with unaffected patients
                    if left_action == self.test_actions[i][0]:
                        score += 1  # Positive reward for each equal pair of actions with unaffected patients
                    # Compare actions for the right wrist
                    if right_action == train_actions_S[i][1]:
                        score += 1  # Positive reward for each equal pair of actions with unaffected patients
                    if right_action == self.test_actions[i][1]:
                        score += 1  # Positive reward for each equal pair of actions with unaffected patients
                    action_scores.append(score)

        # Update the reward
        reward = sum(action_scores)  # Sum of rewards
        
        # Determine episode termination
        done = True  # For simplicity, terminate episode after one step
        
        # Additional info
        info = {}
        
        return self.state, reward, done, info

    def render(self):
        # Visualization code here
        pass

    def reset(self):
        # Reset the environment to an initial state
        self.state = np.array([random.uniform(0, 100) for _ in range(len(self.train_data[0]))])  # Random initial state
        return self.state


In [114]:
env = PatientHealthEnv(train_data, train_actions, test_data, test_actions)


In [116]:
env.observation_space.sample()

array([[59.27754   , 56.019302  , 59.153038  , 91.6612    , 57.72964   ,
        75.85895   ,  5.301171  , 24.665464  , 26.422525  , 30.811039  ,
        54.61951   , 13.099318  , 54.855076  , 83.11588   , 74.264244  ,
         0.20746039, 20.40992   , 77.22429   , 25.903381  , 76.26977   ,
        51.919426  ,  0.20261297, 18.988607  , 33.11864   , 35.29969   ,
         6.0338826 ,  2.1900058 , 28.41636   , 31.398695  , 29.595303  ,
        15.1641865 , 39.672283  , 91.06456   , 14.512331  , 29.126211  ,
        19.054916  , 56.777195  , 27.485245  , 86.39471   , 68.87559   ,
        22.98945   , 41.7827    , 57.959766  ,  1.1493707 , 83.14263   ,
        90.96932   , 26.88445   , 35.3395    , 32.812588  , 91.82547   ,
        83.361176  , 10.483692  , 72.37567   , 69.818436  , 51.29187   ,
        15.9188795 , 31.09968   , 17.864765  , 29.364046  ,  9.992777  ,
        23.156898  , 71.995926  , 20.284693  , 63.636     , 92.692024  ,
         0.20211692,  6.6099577 , 69.108635  , 21.5

In [117]:
env.action_space.sample()

18310

In [118]:
##TEST ENVIRONMENT##
episodes = 5
for episode in range (1, episodes+1):
    obs= env.reset()
    done=False
    score=0

    while not done:
        env.render()
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'. format(episode, score))
#env.close() 

TypeError: cannot unpack non-iterable int object